In [1]:
import scrape

In [ ]:
d = scrape.scrape()

---->     Extracting page:  https://www.jobspider.com/job/view-resume-83965.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83943.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83929.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83923.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83916.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83898.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83896.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83885.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83875.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-83874.html 
    ......
---->     Extracting page:  https://www.jobspider.com/job/view-resume-

In [ ]:
d.keys()

In [ ]:
d['SpiderID']